# Práctica 1 - Clasificación de palabras 

La práctica consiste en entrenar un modelo basado en SVM para clasificar las palabras de un conjunto de datos. La clasificación es entre palabras catalanas o inglesas.

In [3]:
import pandas as pd
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

import numpy as np
from sklearn.svm import SVC

## Preparación de los datos
Primero cargamos el conjunto de datos en el programa usando la libreria pandas. Hemos eliminado previamente la columna de índices, ya que no nos aportaba ningún valor y solo manchaba el dataframe.

In [4]:
df = pd.read_csv("data/data.csv")
print(df)

         catala     angles
0           com         as
1          seva        his
2           que       that
3           ell         he
4           era        was
..          ...        ...
983         nas       nose
984      plural     plural
985      còlera      anger
986  reclamació      claim
987   continent  continent

[988 rows x 2 columns]


El conjunto de datos está constituido por dos columnas, una de palabras en inglés y otra de su traducción en catalan. Para entrenar el modelo vamos a adecuar el formato de las muestras a uno que siga el siguiente formato: C1 | C2 | C3 | .. | CN | Y (dónde C es una característica y Y es la target). El target en este caso será la lengua a la que pertenece la palabra. 

In [5]:
#Funcion que crea las columnas de datos y de resoluciones
def reformat(dataFrame):
    dataFrame['y']=dataFrame.columns[0]
    dataFrame.rename(columns={dataFrame.columns[0]: 'word'}, inplace=True)
    return dataFrame;

#Separamos las palabras por idioma
catala, angles= df.filter(['catala'], axis=1), df.filter(['angles'], axis=1)
#Reformateamos
catala=reformat(catala)
angles=reformat(angles)
#Concatenamos los dos conjuntos de datos
wordsDF=pd.concat([catala,angles], axis=0)
#Mezclamos las filas para que las palabras de cada idioma no se encuentren todas juntas.
wordsDF = wordsDF.sample(frac=1).reset_index(drop=True)
wordsDF.to_csv('data/definitiveData.csv', index=False)
print(wordsDF.to_string)

<bound method DataFrame.to_string of            word       y
0          neta  catala
1      incloure  catala
2     condition  angles
3         ràpid  catala
4          head  angles
...         ...     ...
1971      right  angles
1972      plana  catala
1973    desgast  catala
1974      would  angles
1975      venir  catala

[1976 rows x 2 columns]>


## Características
El siguiente paso es determinar los parámetros sobre los cuales el svm se va a basar para hacer la distinción. Para ello hemos pensado que las siguientes características podrían servirnos para clasificar las palabras:

- Longitud de la palabra (Numérico)
- Ratio de consonantes por vocal (consonantes / vocales) (Númerico)
- Patrones o reglas de ortografía propias de un idioma (Categórico)
    + Acentos, propios del catalán en este dataset
    + Misma vocal concatenada (aa, ee, ii, oo, uu), propia del inglés para representar sonidos
    + Formaciones de consonantes (consonant clusters) para al igual que en el caso de arriba representar sonidos (th, sch)

Referencias
- <Consonant_Clusters: https://www.aprendeinglessila.com/2013/09/consonantes-ingles-clusters/>

Para añadir estas características al dataframe, hemos definido las siguientes funciones: 

In [6]:
#longitud de palabras
def contarLletres(lista):
    cantidadLetras = []
    for word in lista:
        cantidadLetras.append(len(word))
    return cantidadLetras
